In [64]:
import pandas as pd
import spacy
import json
from spacy.training.example import Example
from spacy.tokens import DocBin
from spacy.cli.train import train
from sklearn.model_selection import train_test_split
import os

In [47]:
config_path = "config.cfg"  # Update with your config file path
# Load CSV file
file_path = "master-gdc-gdcdatasets-2020446966_en-2020446966_en\output1.csv"  # Update with your file path

<>:3: SyntaxWarning: invalid escape sequence '\o'
<>:3: SyntaxWarning: invalid escape sequence '\o'
C:\Users\User\AppData\Local\Temp\ipykernel_23144\64075232.py:3: SyntaxWarning: invalid escape sequence '\o'
  file_path = "master-gdc-gdcdatasets-2020446966_en-2020446966_en\output1.csv"  # Update with your file path


In [48]:
def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [49]:
# Function to extract entities using pre-trained spaCy model
def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [50]:
# Apply NER extraction to the description column
df["Extracted_Entities"] = df["description"].apply(extract_entities)

In [51]:
# Convert extracted entities into training format
TRAIN_DATA = []
for _, row in df.iterrows():
    text = row["description"]
    entities = [(text.find(ent[0]), text.find(ent[0]) + len(ent[0]), ent[1]) for ent in row["Extracted_Entities"] if text.find(ent[0]) != -1]
    if entities:
        TRAIN_DATA.append((text, {"entities": entities}))

In [52]:
# Save TRAIN_DATA as JSON for training
train_data_file = "ner_training_data.json"
with open(train_data_file, "w") as f:
    json.dump(TRAIN_DATA, f, indent=4)
print(f"NER training data saved to {train_data_file}")

NER training data saved to ner_training_data.json


In [66]:
# File paths
train_data_path = "train_data.spacy"
dev_data_path = "dev_data.spacy"

# Load blank spaCy model
nlp = spacy.blank("en")  

# Split into training (80%) and validation (20%) data
train_data, dev_data = train_test_split(TRAIN_DATA, test_size=0.2, random_state=42)

# Function to save data to spaCy format
def save_spacy_data(data, file_path):
    db = DocBin()
    for text, annotations in data:
        doc = nlp.make_doc(text)

        # Ensure valid entity formatting
        if "entities" not in annotations or not annotations["entities"]:
            print(f"Skipping invalid entry: {text}")
            continue

        # Remove duplicate & overlapping entities
        unique_entities = list(set(annotations["entities"]))
        unique_entities.sort(key=lambda x: x[0])  

        non_overlapping_entities = []
        last_end = -1
        for start, end, label in unique_entities:
            if start >= last_end:
                non_overlapping_entities.append((start, end, label))
                last_end = end

        annotations["entities"] = non_overlapping_entities

        # Convert to spaCy Example object
        example = Example.from_dict(doc, annotations)
        db.add(example.reference)

    db.to_disk(file_path)

    # Save training and validation data
save_spacy_data(train_data, train_data_path)
save_spacy_data(dev_data, dev_data_path)

c:\Users\User\anaconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "In preparation for the peace conference that was e..." with entities "[(68, 79, 'EVENT'), (84, 102, 'DATE'), (103, 129, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "This panoramic view of Constantinople in 1616 is f..." with entities "[(41, 43, 'CARDINAL'), (125, 132, 'NORP'), (143, 1...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could n

In [67]:
# Train the model using the config file
train(config_path, output_path="trained_ner_model", overrides={"training.max_epochs": 10})
print(f"NER Model training complete! Saved as 'trained_ner_model'")

✔ Created output directory: trained_ner_model
ℹ Saving to output directory: trained_ner_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    161.86    0.00    0.00    0.00    0.00
  0     200       4643.26  12197.21   40.26   53.27   32.35    0.40
  0     400       3113.03   5416.98   62.95   64.59   61.39    0.63
  0     600       2092.95   3938.39   69.45   73.71   65.66    0.69
  0     800       1585.56   4107.48   72.01   70.85   73.20    0.72
  0    1000       5551.11   4042.41   74.59   77.74   71.69    0.75
  0    1200       1183.53   3417.19   74.62   72.75   76.60    0.75
  0    1400       3178.90   3445.01   78.0